## Overview


The goal of this Proof of Concept (PoC) project is to create a tool that do web scraping of Website "https://www.holidify.com/country/india/places-to-visit.html".

### Key Features:


- Scrape tourist places info:

  Extract place names, State, description, Rating out of 5.0, Ranking out of 100, number of tourist attractions and best time to visit.

- Store Data: Save the scraped data to a CSV file or database for future analysis.

In [5]:
#!pip install requests beautifulsoup4 pandas

In [27]:
#importing Libraries
import os
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [7]:
webpage = "https://www.holidify.com/country/india/places-to-visit.html"

response = requests.get(url=webpage)
response.ok

True

In [8]:
inspect_data = BeautifulSoup(response.text,'html.parser')

In [9]:
containers = inspect_data.findAll("div",{"class":"col-12 col-md-6 pr-md-3"})
len(containers)

42

In [10]:

data_list = []

In [11]:
containers[0]

<div class="col-12 col-md-6 pr-md-3">
<div class="card content-card" data-itemid="GULMARG">
<a data-href="/places/gulmarg/" data-position="1" href="/places/gulmarg/">
<h3 class="card-heading"> 1. Gulmarg </h3>
<div class="position-relative">
<div class="collection-scrollable" data-hotel-position="1">
<img alt="" class="card-img-top lazy" data-original="https://www.holidify.com/images/bgImages/GULMARG.jpg" src="/res/images/patt.png"/>
<div class="lazyBG card-img-top" data-original="https://www.holidify.com/images/compressed/dest_wiki_5433.jpg" style="background-image:url('https://holidify.com/images/patt.png');">
</div>
<div class="lazyBG card-img-top" data-original="https://www.holidify.com/images/cmsuploads/compressed/IMG_3768_20190710152743.JPG" style="background-image:url('https://holidify.com/images/patt.png');">
</div>
<div class="lazyBG card-img-top" data-original="https://www.holidify.com/images/cmsuploads/compressed/2771936432_d603c3fbd9_b_20190710152801.jpg" style="background-

In [12]:
for container in containers:
    data = {}
    pname = container.find("h3", {"class":"card-heading"} )
    data['p_name'] = pname.get_text().split('.')[1].split('-')[0].strip()

    rating = container.find("span",{"class":"rating-badge"})
    if rating:
        rating_val = rating.find('b').get_text().strip()
        rating_max = rating.find('span',{"class":"light"}).get_text().strip()
        rating = rating_val+rating_max
    else: rating = 0
    data['p_rating'] = rating
    
    objective = container.find("p", {"class":{"objective"}}).get_text().strip()
    data['rank'] = objective.split('Places to visit in India')[0]
    if objective.split('Places to visit in India')[1]:
        data['tourist_attractions'] = objective.split('Places to visit in India')[1].strip().split()[0]
    else: data['tourist_attractions'] =''

    mb = container.find("p", {"class":{"mb-2"}})
    if mb:
        data['location'] = mb.find('a').get_text().strip()
    else:data['location'] =''

    data['description'] = container.find('p',{'class':'card-text'}).get_text().strip()

    mb_3 = container.findAll('p',class_='mb-3')
    for mb in mb_3:
        if mb.find('b').get_text().strip()=='Known For :':
            mb3 = mb.findAll("span",{"class": "clickable align-middle"})
            known_for=[]
            for span in mb3:
                known_for.append(span.find('b').get_text().strip())
            data['known_for'] = known_for
        elif "Best Time:" in mb.get_text():
            data['best_time'] = mb.get_text().split("Best Time: ")[len(mb.get_text().split("Best Time: "))-1]
    data_list.append(data)

In [13]:
result = pd.DataFrame(data_list,columns = ['p_name','p_rating','rank','tourist_attractions','location','description','known_for','best_time'])


In [55]:
result.head(5)

,p_name,p_rating,rank,tourist_attractions,location,description,known_for,best_time
0,Gulmarg,4.4/5,1 out of 100,29,Jammu and Kashmir,Situated at an altitude of 2730 m above sea le...,"[Gulmarg Gondola, Alpather Lake, Apharwat Peak]",October to June
1,Munnar,4.2/5,2 out of 100,54,Kerala,"Famous for the tea estates, greenery, winding ...","[Mattupetty Dam, Kolukkumalai Tea Estate, Rose...",September to May
2,Gangtok,4.6/5,3 out of 100,34,Sikkim,"Incredibly alluring, pleasantly boisterous and...","[Nathula Pass, MG Marg, Rumtek Monastery]",Throughout the year
3,Manali,4.8/5,4 out of 100,53,Himachal Pradesh,"With spectacular valleys, breathtaking views, ...","[Solang Valley, Hadimba Devi Temple, Manali Ma...",October to June
4,Srinagar,4.2/5,5 out of 100,59,Jammu and Kashmir,"Famously known as 'Heaven on Earth, Srinagar i...","[Dal Lake, Indira Gandhi Memorial Tulip Garden...",April to October


In [53]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   p_name               42 non-null     object
 1   p_rating             42 non-null     object
 2   rank                 42 non-null     object
 3   tourist_attractions  42 non-null     object
 4   location             42 non-null     object
 5   description          42 non-null     object
 6   known_for            40 non-null     object
 7   best_time            42 non-null     object
dtypes: object(8)
memory usage: 2.8+ KB


In [51]:
parent_dir = os.getcwd()
result.to_csv(os.path.join(parent_dir,'tourist_places.csv'),index=False)
print('Stored the Results to CSV file')

Stored the Results to CSV file
